In [3]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local[2]"))

In [4]:
import re

In [5]:
def parse_article(line):
    try:
        article_id, text = unicode(line.rstrip()).split('\t', 1)
    except ValueError as e:
        return []
    text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    return words

In [6]:
wiki = sc.textFile("/data/wiki/en_articles_part/articles-part", 16).map(parse_article)

In [7]:
result = wiki.take(1)[0]

In [8]:
for word in result[:50]:
    print word

Anarchism
Anarchism
is
often
defined
as
a
political
philosophy
which
holds
the
state
to
be
undesirable
unnecessary
or
harmful
The
following
sources
cite
anarchism
as
a
political
philosophy
Slevin
Carl
Anarchism
The
Concise
Oxford
Dictionary
of
Politics
Ed
Iain
McLean
and
Alistair
McMillan
Oxford
University
Press
2003
However
others
argue


In [9]:
def search_key(words):
    pairs = []
    for i,word in enumerate(words[:-1]):
        if word == "narodnaya":
            pair = word+"_"+words[i+1]
            pairs.append((pair,1))
        else:
            continue
    return pairs

In [10]:
lower_words = wiki.map(lambda words:[x.lower() for x in words])

In [11]:
word_pairs = lower_words.flatMap(lambda x : search_key(x))

In [12]:
reduce_word = word_pairs.reduceByKey(lambda x, y: x + y)

In [13]:
sorted_words = reduce_word.sortByKey()

In [14]:
result = sorted_words.collect()

In [16]:
for pair, cnt in result:
    print '{}\t{}'.format(pair, cnt)

narodnaya_gazeta	1
narodnaya_volya	9
